In [38]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

import os
print(os.listdir("/storage/santander_comp/"))

['.ipynb_checkpoints', 'train_aug.csv', 'submissions', 'fake_test.csv', 'test_aug.csv', 'test.csv', 'models', 'real_test.csv', 'train.csv', 'testIDs.csv', 'nbs']


In [43]:
test_path = "/storage/santander_comp/test.csv"

df_test = pd.read_csv(test_path)
df_test.drop(['ID_code'], axis = 1, inplace = True)
df_test = df_test.values

In [7]:
train_path = '/storage/santander_comp/train.csv'

df_train = pd.read_csv(train_path)
df_train.drop(['ID_code', 'target'], axis = 1, inplace = True)
df_train = df_train.values

In [44]:
def uniques(df):
    unique_samples = []
    unique_count = np.zeros_like(df_test)
    for feature in tqdm(range(df.shape[1])):
        _, index_, count_ = np.unique(df[:, feature], 
                                      return_counts=True, 
                                      return_index=True)
        unique_count[index_[count_ == 1], feature] += 1
    return unique_count

In [9]:
uniques_train = uniques(df_train)

In [45]:
uniques_test = uniques(df_test)

In [46]:
real_samples_indexes_test = np.argwhere(np.sum(uniques_test, axis = 1) > 0)[:, 0]
synthetic_samples_indexes_test = np.argwhere(np.sum(uniques_test, axis = 1) == 0)[:, 0]

In [47]:
df_test = pd.read_csv(test_path)

In [48]:
test_real = df_test.loc[real_samples_indexes_test].copy()

In [49]:
test_fake = df_test.loc[synthetic_samples_indexes_test].copy()

In [50]:
test_real.to_csv('real_test.csv', index=False)
test_fake.to_csv('fake_test.csv', index=False)

In [13]:
real_samples_indexes_test = np.argwhere(np.sum(uniques_test, axis = 1) > 0)[:, 0]
real_samples_indexes_train = np.argwhere(np.sum(uniques_train, axis = 1) > 0)[:, 0]
synthetic_samples_indexes_test = np.argwhere(np.sum(uniques_test, axis = 1) == 0)[:, 0]
synthetic_samples_indexes_train = np.argwhere(np.sum(uniques_train, axis = 1) == 0)[:, 0]

In [14]:
print(len(real_samples_indexes_test))
print(len(synthetic_samples_indexes_test))
print(len(real_samples_indexes_train))
print(len(synthetic_samples_indexes_train))

100000
100000
200000
0


In [21]:
test_path = "/storage/santander_comp/test.csv"

df_test = pd.read_csv(test_path)
df_test.drop(['ID_code'], axis = 1, inplace = True)

In [24]:
df_test.drop(['ID_code'], axis = 1, inplace = True)

In [25]:
df_test.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [30]:
real_samples_indexes_test

array([     3,      7,     11, ..., 199995, 199996, 199999])

In [33]:
df_test_real = df_test.loc[real_samples_indexes_test].copy()

In [34]:
df_test_real.shape

(100000, 200)

In [35]:
df_test_fake = df_test.loc[synthetic_samples_indexes_test].copy()

In [36]:
df_test_fake.shape

(100000, 200)

In [37]:
df_test_real.to_csv('real_test.csv', index=False)
df_test_fake.to_csv('fake_test.csv', index=False)

In [16]:
df_test_real = df_test[real_samples_indexes_test].copy()

generator_for_each_synthetic_sample = []
# Using 20,000 samples should be enough. 
# You can use all of the 100,000 and get the same results (but 5 times slower)
for cur_sample_index in tqdm(synthetic_samples_indexes_test[:20000]):
    cur_synthetic_sample = df_test[cur_sample_index]
    potential_generators = df_test_real == cur_synthetic_sample

    # A verified generator for a synthetic sample is achieved
    # only if the value of a feature appears only once in the
    # entire real samples set
    features_mask = np.sum(potential_generators, axis=0) == 1
    verified_generators_mask = np.any(potential_generators[:, features_mask], axis=1)
    verified_generators_for_sample = real_samples_indexes_test[np.argwhere(verified_generators_mask)[:, 0]]
    generator_for_each_synthetic_sample.append(set(verified_generators_for_sample))

In [17]:
public_LB = generator_for_each_synthetic_sample[0]
for x in tqdm(generator_for_each_synthetic_sample):
    if public_LB.intersection(x):
        public_LB = public_LB.union(x)

private_LB = generator_for_each_synthetic_sample[1]
for x in tqdm(generator_for_each_synthetic_sample):
    if private_LB.intersection(x):
        private_LB = private_LB.union(x)
        
print(len(public_LB))
print(len(private_LB))


50000
50000


In [18]:
public_LB

{131076,
 131078,
 7,
 131080,
 131081,
 131083,
 17,
 18,
 131092,
 21,
 131095,
 24,
 131099,
 131100,
 29,
 131101,
 131106,
 131108,
 131110,
 131111,
 131114,
 131115,
 48,
 131134,
 63,
 68,
 69,
 131146,
 77,
 79,
 131151,
 81,
 131154,
 131162,
 91,
 92,
 131164,
 131166,
 131168,
 97,
 131169,
 131170,
 131172,
 131174,
 110,
 131184,
 112,
 131186,
 115,
 131188,
 114,
 120,
 121,
 131194,
 122,
 124,
 131196,
 127,
 131203,
 132,
 131205,
 135,
 138,
 140,
 131215,
 144,
 145,
 131217,
 131221,
 149,
 131230,
 131231,
 131238,
 172,
 173,
 131248,
 177,
 131250,
 180,
 131252,
 131253,
 131256,
 184,
 131257,
 131259,
 187,
 189,
 191,
 131265,
 131269,
 199,
 203,
 131276,
 131281,
 131283,
 131284,
 131285,
 131289,
 131290,
 220,
 223,
 131297,
 131299,
 228,
 131304,
 233,
 236,
 131311,
 240,
 239,
 242,
 243,
 131316,
 131317,
 245,
 247,
 131320,
 248,
 131321,
 253,
 131331,
 259,
 131333,
 131334,
 131335,
 131338,
 267,
 131343,
 131347,
 131348,
 278,
 280,
 282,


In [12]:
np.save('public_LB', list(public_LB))
np.save('private_LB', list(private_LB))
np.save('synthetic_samples_indexes', list(synthetic_samples_indexes))